In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import os
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#Dowload datasets ecommerce-dataset from https://www.kaggle.com/retailrocket/ecommerce-dataset to DIR
DIR='/Users/forken/tensorflow_datasets/ecommerce-dataset'
for dirname, _, filenames in os.walk(DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/Users/forken/tensorflow_datasets/ecommerce-dataset/item_properties_part1.csv
/Users/forken/tensorflow_datasets/ecommerce-dataset/item_properties_part2.csv
/Users/forken/tensorflow_datasets/ecommerce-dataset/category_tree.csv
/Users/forken/tensorflow_datasets/ecommerce-dataset/events.csv


In [130]:
events = pd.read_csv(DIR+"/events.csv")

In [23]:
events.shape

(2756101, 5)

In [24]:
events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [131]:
events['date']=events.timestamp.map(lambda ts: datetime.utcfromtimestamp(ts/1000.0).strftime('%Y-%m-%d'))
events=events.sort_values(by='timestamp').reset_index(drop=True)
events.head()

,timestamp,visitorid,event,itemid,transactionid,date
0,1430622004384,693516,addtocart,297662,NaN,2015-05-03
1,1430622011289,829044,view,60987,NaN,2015-05-03
2,1430622013048,652699,view,252860,NaN,2015-05-03
3,1430622024154,1125936,view,33661,NaN,2015-05-03
4,1430622026228,693516,view,297662,NaN,2015-05-03


In [33]:
events.event.unique()

array(['view', 'addtocart', 'transaction'], dtype=object)

In [132]:
liked_items=events[events.event!='view'][['visitorid','itemid']].drop_duplicates()
liked_items['liked']=1
liked_items.head()

,visitorid,itemid,liked
0,693516,297662,1
26,979664,338222,1
69,260113,125751,1
156,319455,342530,1
174,345781,438400,1


In [133]:
events=events.merge(liked_items,'outer',['visitorid','itemid'],suffixes=(False,False))
events['liked'] = events['liked'].apply(lambda x: 1 if x==1 else 0)

In [134]:
events=events[["timestamp","visitorid","itemid","liked","date"]]

In [201]:
A=events.merge(events,on=['visitorid'],suffixes=("","_pre"))

In [202]:
B=A[(A.timestamp > A.timestamp_pre) & (((A.timestamp - A.timestamp_pre)/1000.0) < 86400)]

In [203]:
B=B[B.itemid != B.itemid_pre]

In [204]:
B=B.drop_duplicates(subset=["timestamp","visitorid","itemid","itemid_pre"],keep='last')

In [205]:
B=B.groupby(["timestamp","visitorid","itemid"]).tail(20)

In [206]:
B["likedsum_pre"]=B.groupby(["timestamp","visitorid","itemid"])["liked_pre"].transform("sum")
B["itemcnt_pre"]=B.groupby(["timestamp","visitorid","itemid"])["itemid_pre"].transform("count")

In [208]:
B = B[B.likedsum_pre>0]

In [209]:
B.head(10)

,timestamp,visitorid,itemid,liked,date,timestamp_pre,itemid_pre,liked_pre,date_pre,likedsum_pre,itemcnt_pre
293,1430622110079,260113,51983,0,2015-05-03,1430622067992,125751,1,2015-05-03,1,1
360,1438657139006,90447,365821,0,2015-08-04,1438655674315,435047,0,2015-08-04,1,2
361,1438657139006,90447,365821,0,2015-08-04,1438656252819,378505,1,2015-08-04,1,2
367,1438657311855,90447,124455,0,2015-08-04,1438655674315,435047,0,2015-08-04,1,3
368,1438657311855,90447,124455,0,2015-08-04,1438656252819,378505,1,2015-08-04,1,3
370,1438657311855,90447,124455,0,2015-08-04,1438657139006,365821,0,2015-08-04,1,3
395,1430622154356,979664,91998,0,2015-05-03,1430622085008,338222,1,2015-05-03,1,1
406,1430622164188,979664,355594,0,2015-05-03,1430622085008,338222,1,2015-05-03,1,2
408,1430622164188,979664,355594,0,2015-05-03,1430622154356,91998,0,2015-05-03,1,2
417,1430622591111,979664,355594,0,2015-05-03,1430622085008,338222,1,2015-05-03,1,5


In [323]:
def f(x):
    y=x.iloc[::-1]
    n2pad = 20 - x.shape[0]
    d = {}
    d['itemid_pre'] = np.pad(y['itemid_pre'].to_numpy(dtype='int64'),(0,n2pad),'constant',constant_values=(0,0))
    d['liked_pre'] = np.pad(y['liked_pre'].to_numpy(dtype='int64'),(0,n2pad),'constant',constant_values=(0,0))
    d['timestamp_pre'] = np.pad(y['timestamp_pre'].to_numpy(dtype='int64'),(0,n2pad),'constant',constant_values=(0,0))
    return pd.Series(d, index=['itemid_pre', 'liked_pre','timestamp_pre'])

In [353]:
data = B.groupby(["liked","timestamp","visitorid","itemid","date","itemcnt_pre","likedsum_pre"]).apply(f)

In [354]:
data = data.reset_index()

In [355]:
def push_zeros_back(a):
    """Return a copy of the iterable a with all zeros at the end."""
    return np.array(sorted(a, key=lambda x: x == 0))

In [358]:
data["itemid_pre_liked"] = (data["itemid_pre"]*data["liked_pre"])
data["itemid_pre_nliked"] = (data["itemid_pre"]*(1-data["liked_pre"]))

In [359]:
data.head(30)

,liked,timestamp,visitorid,itemid,date,itemcnt_pre,likedsum_pre,itemid_pre,liked_pre,timestamp_pre,itemid_pre_liked,itemid_pre_nliked
0,0,1430622110079,260113,51983,2015-05-03,1,1,"[125751, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622067992, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[125751, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0,1430622154356,979664,91998,2015-05-03,1,1,"[338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622085008, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0,1430622164188,979664,355594,2015-05-03,2,1,"[91998, 338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622154356, 1430622085008, 0, 0, 0, 0, 0, ...","[0, 338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[91998, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,0,1430622231863,979664,430965,2015-05-03,3,1,"[355594, 91998, 338222, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622164188, 1430622154356, 1430622085008, ...","[0, 0, 338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[355594, 91998, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0,1430622312158,979664,454393,2015-05-03,4,1,"[430965, 355594, 91998, 338222, 0, 0, 0, 0, 0,...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622231863, 1430622164188, 1430622154356, ...","[0, 0, 0, 338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[430965, 355594, 91998, 0, 0, 0, 0, 0, 0, 0, 0..."
5,0,1430622459202,979664,54805,2015-05-03,5,1,"[454393, 430965, 355594, 91998, 338222, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622312158, 1430622231863, 1430622164188, ...","[0, 0, 0, 0, 338222, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[454393, 430965, 355594, 91998, 0, 0, 0, 0, 0,..."
6,0,1430622591111,979664,355594,2015-05-03,5,1,"[54805, 454393, 430965, 91998, 338222, 0, 0, 0...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622459202, 1430622312158, 1430622231863, ...","[0, 0, 0, 0, 338222, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[54805, 454393, 430965, 91998, 0, 0, 0, 0, 0, ..."
7,0,1430622706111,979664,162756,2015-05-03,6,1,"[54805, 454393, 430965, 355594, 91998, 338222,...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622459202, 1430622312158, 1430622231863, ...","[0, 0, 0, 0, 0, 338222, 0, 0, 0, 0, 0, 0, 0, 0...","[54805, 454393, 430965, 355594, 91998, 0, 0, 0..."
8,0,1430622950792,979664,117649,2015-05-03,7,1,"[162756, 54805, 454393, 430965, 355594, 91998,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622706111, 1430622459202, 1430622312158, ...","[0, 0, 0, 0, 0, 0, 338222, 0, 0, 0, 0, 0, 0, 0...","[162756, 54805, 454393, 430965, 355594, 91998,..."
9,0,1430622964150,1375898,357854,2015-05-03,2,1,"[461547, 64152, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622912969, 1430622790487, 0, 0, 0, 0, 0, ...","[461547, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 64152, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [360]:
data.to_pickle('ecommerce_cvr.pkl',protocol=2)
data2 = pd.read_pickle('ecommerce_cvr.pkl')

In [361]:
data2.head()

,liked,timestamp,visitorid,itemid,date,itemcnt_pre,likedsum_pre,itemid_pre,liked_pre,timestamp_pre,itemid_pre_liked,itemid_pre_nliked
0,0,1430622110079,260113,51983,2015-05-03,1,1,"[125751, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622067992, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[125751, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0,1430622154356,979664,91998,2015-05-03,1,1,"[338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622085008, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0,1430622164188,979664,355594,2015-05-03,2,1,"[91998, 338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622154356, 1430622085008, 0, 0, 0, 0, 0, ...","[0, 338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[91998, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,0,1430622231863,979664,430965,2015-05-03,3,1,"[355594, 91998, 338222, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622164188, 1430622154356, 1430622085008, ...","[0, 0, 338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[355594, 91998, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0,1430622312158,979664,454393,2015-05-03,4,1,"[430965, 355594, 91998, 338222, 0, 0, 0, 0, 0,...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1430622231863, 1430622164188, 1430622154356, ...","[0, 0, 0, 338222, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[430965, 355594, 91998, 0, 0, 0, 0, 0, 0, 0, 0..."
